<a href="https://colab.research.google.com/github/ufbfung/mimic/blob/main/mimic_cxr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MIMIC-CXR
This notebook will explore the new chest x-ray repository from mimic.

## Setup environment


### Mount Google Drive & Map File Paths
This section will connect to your google drive and define any file paths for subsequent access and references.

In [10]:
# Import relevant libraries
from google.colab import drive # to mount google drive
import os # to deal with file paths
import numpy as np # to deal with data
import pandas as pd # to deal with data
import cv2 # to read videos
from matplotlib import pyplot as plt # to plot figures

# Mount drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Define file paths
root_path = '/drive/My Drive/coding/mimic/'

os.listdir(root_path)

FileNotFoundError: ignored

### Integrate with BigQuery
This section will allow integration with Google BigQuery to bring data into your Colab instance.

### Authenticate and authorize Colab Notebook
This step will install relevant libraries for authentication and authorization and also authenticate your Colab notebook to connect to Google BigQuery.

In [ ]:
# Install relevant libraries
from google.colab import auth # this provides functions for authenticating and authorizing your Google account within the Colab environment

# Authenticate
auth.authenticate_user() # this initiates the authentication process which will prompt you to authenticate your Google account. The purpose of this authentication is to allow your Colab notebook to access Google Cloud services, such as BigQuery, using your authorized account.

### Create a client
This step will create a client to access a specific project within BigQuery. You will want to copy the name of the project you're interested in querying from BigQuery and set it to the project variable.

In [ ]:
# Install relevant library
from google.cloud import bigquery

# Define project you're interested in querying from BigQuery
project = 'mimic-iv-390722'

# Create a client
client = bigquery.Client(project=project)

### Write a query
This step involves writing an SQL query you want to use. Below is a simple query that can be used to quickly check the connection

In [ ]:
# Write a sample query to check connection
query = """
SELECT *
FROM `physionet-data.mimiciii_clinical.admissions`
LIMIT 5
"""

### Run the query & get results


In [ ]:
# Run the query
query_job = client.query(query)

# Get the results
results = query_job.result()

### Process and extract the data
This will process the data from the query and print them

In [ ]:
# Process and extract the data
for row in results:
  print(row) # process the row data

Row((3757, 3115, 134067, datetime.datetime(2139, 2, 13, 3, 11), datetime.datetime(2139, 2, 20, 7, 33), None, 'EMERGENCY', 'EMERGENCY ROOM ADMIT', 'SNF', 'Medicare', None, None, None, 'WHITE', datetime.datetime(2139, 2, 13, 0, 2), datetime.datetime(2139, 2, 13, 3, 22), 'STAB WOUND', 0, 1), {'ROW_ID': 0, 'SUBJECT_ID': 1, 'HADM_ID': 2, 'ADMITTIME': 3, 'DISCHTIME': 4, 'DEATHTIME': 5, 'ADMISSION_TYPE': 6, 'ADMISSION_LOCATION': 7, 'DISCHARGE_LOCATION': 8, 'INSURANCE': 9, 'LANGUAGE': 10, 'RELIGION': 11, 'MARITAL_STATUS': 12, 'ETHNICITY': 13, 'EDREGTIME': 14, 'EDOUTTIME': 15, 'DIAGNOSIS': 16, 'HOSPITAL_EXPIRE_FLAG': 17, 'HAS_CHARTEVENTS_DATA': 18})
Row((8689, 7124, 109129, datetime.datetime(2188, 7, 11, 0, 58), datetime.datetime(2188, 8, 1, 12, 4), None, 'EMERGENCY', 'EMERGENCY ROOM ADMIT', 'SNF', 'Medicare', None, None, None, 'WHITE', datetime.datetime(2188, 7, 10, 14, 17), datetime.datetime(2188, 7, 11, 1, 52), 'PENILE LACERATION-CELLULITIS', 0, 1), {'ROW_ID': 0, 'SUBJECT_ID': 1, 'HADM_ID': 

## Connect to Google Cloud Storage (GCS) to access the chest x-rays
We will plan to access the images directly from GCS vs. uploading them. This section will access the GCS bucket to process the images.

This section will assume that you have already authenticated with Google Cloud from the prior step using auth.authenticate_user()

In [ ]:
# Install relevant libraries
from google.cloud import storage

# Create a client
project_id = 'mimic-iv-390722'
storage_client = storage.Client(project=project_id)

# Get a bucket reference
bucket_name = 'mimic-iv-cxr'
bucket = storage_client.get_bucket(bucket_name)

# List objects in the bucket
blobs = bucket.list_blobs()

# Set paths to files
# files_path =

# Alternatively, you can directly access a specific blob by its name
specific_blob = bucket.blob('files/p10/p10000032/s50414267/02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.jpg')
print(specific_blob)
# Perform operations on the specific blob
# For example, download the file
#specific_blob.download_to_filename('local_file.jpg')
#print("File downloaded successfully.")
#for blob in blobs:
#  print(blob.name)

<Blob: mimic-iv-cxr, files/p10/p10000032/s50414267/02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.jpg, None>


## Setup ResNet CNN
- For the training-validate-test splits, we'll use the predefined studies set by mimic in their mimic-cxr-2.0.0-split.csv

In [ ]:
#

# Explore the data
Once everything is setup. We can start to run some queries to get an idea of what the dataset looks like. We'll start with some queries that profile the MIMIC-III dataset.

In this section, we'll also define some helper functions that will assist us in querying the data.

A examplar notebook for exploring the data will be primarily based on the [MIMIC code repository](https://github.com/MIT-LCP/mimic-code/blob/main/mimic-iv/notebooks/tableone.ipynb) for doing just that - exploring demographics and charts within MIMIC-IV.

In [ ]:
# !pip install tableone
# from tableone import TableOne

In [ ]:
# Import relevant libraries for exploration
from collections import OrderedDict
from tabulate import tabulate
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# Define helper functions
def query_dataset(query):
  # Run the query
  query_job = client.query(query)

  # Get the results
  results = query_job.result()

  # Process and extract the data
  for row in results:
    print(row) # process the row data

def create_query(dataset, table):
  project = 'physionet-data'
  query = """
  SELECT *
  FROM `{0}.{1}.{2}`
  LIMIT 3
  """.format(project, dataset, table)
  return query

def summarize_data(dataset, table):
    # Run the query to retrieve the data
    query = """
    SELECT *
    FROM `{0}.{1}.{2}`
    """.format('physionet-data', dataset, table)

    query_job = client.query(query)
    results = query_job.result()

    # Initialize counters and dictionaries
    total_patients = 0
    distinct_patients = set()
    disease_counts = {}
    disease_percentages = {}

    # Process and extract the data
    for row in results:
        total_patients += 1
        distinct_patients.add(row.subject_id)

        # Iterate over the diseases/findings columns
        for column_name in row.keys():
            if column_name not in ['subject_id', 'study_id']:
                disease_value = row[column_name]

                # Count the occurrences of each disease/finding
                if disease_value is not None:
                    if column_name not in disease_counts:
                        disease_counts[column_name] = 0
                    disease_counts[column_name] += 1

    # Calculate the percentages of each disease/finding
    for column_name, count in disease_counts.items():
        percentage = (count / total_patients) * 100
        disease_percentages[column_name] = percentage

    # Prepare the summary statistics as a list of lists
    summary_data = [
        ["Total count of patients:", total_patients],
        ["Count of distinct patients:", len(distinct_patients)],
        ["Count of each disease/finding (from most frequent to least frequent):"]
    ]
    for column_name, count in sorted(disease_counts.items(), key=lambda x: x[1], reverse=True):
        summary_data.append([column_name, count])
    summary_data.append(["Percentage of each disease/finding to the total count of patients:"])
    for column_name, percentage in sorted(disease_percentages.items(), key=lambda x: x[1], reverse=True):
        summary_data.append([column_name, "{:.2f}%".format(percentage)])

    # Generate the formatted table
    table_output = tabulate(summary_data, headers=["Category", "Count/Percentage"], tablefmt="github")

    # Print the formatted table
    print(table_output)

# Define the dataset and table of interest
dataset = 'mimic_cxr'
table = 'chexpert'

# Create query
# query = create_query(dataset, table)
# query_dataset(query)

# Summarize findings
summarize_data(dataset, table)

| Category                                                              | Count/Percentage   |
|-----------------------------------------------------------------------|--------------------|
| Total count of patients:                                              | 227827             |
| Count of distinct patients:                                           | 65379              |
| Count of each disease/finding (from most frequent to least frequent): |                    |
| Pleural_Effusion                                                      | 87272              |
| No_Finding                                                            | 75455              |
| Support_Devices                                                       | 70281              |
| Cardiomegaly                                                          | 66799              |
| Edema                                                                 | 65833              |
| Pneumonia                                       